# European Options

In [1]:
import yfinance as yf 
import numpy as np 
from scipy.stats import norm

In [2]:
ticker = 'NVDA'
data = yf.download(ticker, period='10y', interval='1d', auto_adjust=True, progress=False)
data['returns'] = np.log(data.Close / data.Close.shift(1))
data.dropna(inplace=True)
data.tail()

,Open,High,Low,Close,Volume,returns
Date,,,,,,
2022-02-16,262.589996,265.820007,255.520004,265.109985,73267600,0.000604
2022-02-17,256.299988,257.850006,241.649994,245.070007,81059500,-0.078601
2022-02-18,246.679993,249.860001,231.000000,236.419998,75966400,-0.035934
2022-02-22,230.350006,240.639999,230.000000,233.899994,63342200,-0.010716
2022-02-23,238.020004,241.550003,223.009995,223.869995,56466000,-0.043828


In [3]:
S_0 = data.Close[-1]
sigma = data.returns.std() * np.sqrt(252)
num_sim = 1_000_000
T = 1 # 1-year
N = 252 # 252 trading days
K = 220 # strike price
rf = 0.01 # risk-free rate

## Black-Scholes formula

$$ C(S_t,T) = N(d_1)S_t - N(d_2) K e^{-r(T-t)} $$ </p>
$$ P(S_t,T) = N(-d_2)K e^{-r(T-t)} - N(-d_1)S_t $$ </p>
$$ d_1 = \frac{1}{\sigma\sqrt{T-t}} [ln(\frac{S_t}{K}) + (r + \frac{\sigma^2}{2})(T-t)] $$
$$ d_2 = d_1 - \sigma \sqrt{T-t} $$

In [4]:
def black_scholes_european(S_0, K, r, sigma, T, option_type='call'):
    d1 = (np.log(S_0/K) + (r + (sigma**2)/2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type.lower() == 'call':
        N_d1 = norm.cdf(d1, loc=0, scale=1)
        N_d2 = norm.cdf(d2, loc=0, scale=1)
        val = (N_d1*S_0) - (N_d2*K*np.exp(-r*T))
    elif option_type.lower() == 'put':
        N_neg_d1 = norm.cdf(-d1, loc=0, scale=1)
        N_neg_d2 = norm.cdf(-d2, loc=0, scale=1)
        val = N_neg_d2*K*np.exp(-r*T) - N_neg_d1*S_0
    else:
        raise ValueError('Wrong option type.')
        
    return val 

In [5]:
call_bs = black_scholes_european(S_0, K, rf, sigma, T, 'call')
put_bs = black_scholes_european(S_0, K, rf, sigma, T, 'put')

In [6]:
print("Black-Scholes Formula")
print("-"*30)
print(f'Strike price: {K}')
print(f'Call premium: {call_bs:.2f}')
print(f'Put premium: {put_bs:.2f}')

Black-Scholes Formula
------------------------------
Strike price: 220
Call premium: 39.07
Put premium: 33.01


## Monte Carlo Simulation

In [7]:
def monte_carlo_european(S_0, K, rf, sigma, num_sim, T, N, option_type='call'):
    rv = np.random.normal(loc=0, scale=1, size=num_sim)
    S_T = S_0 * np.exp((rf - 0.5*(sigma**2)) * T + sigma * np.sqrt(T) * rv) 
    
    if option_type.lower() == 'call':
        payoff = np.maximum(0, S_T - K)
    elif option_type.lower() == 'put':
        payoff = np.maximum(0, K - S_T)
    else:
        raise ValueError('Wrong option type.')
    
    discount_factor = np.exp(-rf * T)
    premium = np.mean(payoff) * discount_factor
    
    return premium

In [8]:
call_mc = monte_carlo_european(S_0, K, rf, sigma, num_sim, T, N, 'call')
put_mc = monte_carlo_european(S_0, K, rf, sigma, num_sim, T, N, 'put')

In [9]:
print("Monte Carlo Simulations")
print("-"*30)
print(f'Strike price: {K}')
print(f'Call premium: {call_mc:.2f}')
print(f'Put premium: {put_mc:.2f}')

Monte Carlo Simulations
------------------------------
Strike price: 220
Call premium: 39.21
Put premium: 32.97
